# Advent of Code Problems

### Key Concepts & Packages

The following packages and ideas are part of the python language itself. They are always available and understanding them will make your life much easier. Individually, they help solve simple problems in the most efficient way, allowing you to combine them to solve more complex problems

  * [Truthy and Falsy values](https://www.freecodecamp.org/news/truthy-and-falsy-values-in-python/)
  * [Indexing](https://towardsdatascience.com/the-basics-of-indexing-and-slicing-python-lists-2d12c90a94cf) and [numpy indexing](https://numpy.org/doc/stable/reference/arrays.indexing.html)
  * Regular expressions [re](https://docs.python.org/3/library/re.html)
  * Hashes and [Collections](https://docs.python.org/3/library/collections.html)


These modules aren't part of the core python language, but they're well written and very widely used.

* [Requests](https://docs.python-requests.org/en/master/) HTTP for humans
* [Pandas](https://pandas.pydata.org/)
* [Numpy](https://numpy.org/)

There are **lots** of other python modules available as well. In general you should look for an existing python module before writing something from scratch, it'll save you a lot of work and let you focus on the interesting parts of your problem. [PyPI](https://pypi.org/) is the canonical source for many packages and integrates with the `pip` package manager. When you are looking at a new package, some of the things to consider are

  1. Does it do what you need? Does it do it well?
  1. Does it have a good support community and active development? Often the associated GitHub issue queue can help you judge this
  
  


## Problem 1: Password Checking

### Part 1

 * N.B. This problem is a tweak of [Advent of Code 2020 problem 2](https://adventofcode.com/2020/day/2)
You've been given the password file for a group of users and you've been asked to validate the entries. Here are some sample entries

```
1-9 a: asjuaycbnwo
3-7 c: qohnfbyvdvxb
5-7 j: abnkjahajjay
3-6 p: papvmnnapa
1-5 n: nannanuuanyn
```

The second field with the colon represents a character that we expect to find in the password (the 3rd field). The first field represents the minimum and maximum number of occurrences of that character which are needed to satisfy our policy. For example, the first line says that the password is `asjuaycbnwo` and that we expect to find at least one to at most nine occurrences of the latter `a` in that password. Given those rules we have

```
1-9 a: asjuaycbnwo   # VALID   2 occurences of "a" (1 <= 2 <= 9)
3-7 c: qohnfbyvdvxb  # INVALID 0 occurences of "c" (0 < 3)
5-7 j: abnkhajajjay  # INVALID 3 occurences of "j" (3 < 5)
3-6 p: papvmnnapa    # VALID   3 occurences of "p" (3<= 3 <= 6)
1-5 n: nannnnuuanyn  # INVALID 7 occurences of "n" (7 > 5)
```

You are asked to calculate the total number of valid passwords in the file.

### Part 1 Answer

* Ingest the data & split it into lines
* Extract password, target character, min and max occurences from each line
* Validate the password using these paramters. Use Regular Expressions if you can!

In [1]:
import re
import requests

try:
    r = requests.get('https://m2pi.syzygy.ca/data/day_2.txt')
    r.raise_for_status()
except Exception as err:
    print(f'Unable to retrieve passwords: {err}')

lines = r.text.strip().split('\n')
lines[:10]

['15-19 k: kkkkkkkkkkkkzkkkkkkk',
 '1-11 s: sbssswsqsssssrlss',
 '8-9 b: pbbbbbbkbz',
 '4-10 w: wwccwcqwdmbktjrxhw',
 '1-6 x: jvscgqsnt',
 '1-7 x: xxxxxxcx',
 '6-10 s: smssssfskssdwvtcss',
 '6-12 q: qqqqzqqjqfqdqq',
 '3-7 d: ddwbzbf',
 '12-14 s: ssdssssssssmsq']

In [2]:
pwre = re.compile(
    '(?P<chmin>\d+)-(?P<chmax>\d+) (?P<ch>[a-z]): (?P<pw>[a-z]+)'
)

valid_pw = 0
for line in lines:
    m = pwre.match(line).groupdict()
    m['chmin'], m['chmax'] = int(m['chmin']), int(m['chmax'])
    occurrences = len(re.findall(m['ch'], m['pw']))
    if (m['chmin'] <= occurrences <= m['chmax']):
        valid_pw += 1

print(f"{valid_pw} VALID passwords ({len(lines) - valid_pw} INVALID)")

655 VALID passwords (345 INVALID)


### Part 2

Part two of this problem says that the schema was misinterpreted in part 1. Actually, the first token on each line should be interpretated as positions in the password string (indexed from 1!) and that the given character should occur **exactly** once in those positions. Recycling our examples from above

```
1-9 a: asjuaycbnwo  # VALID "a" occurs in position 1
3-7 c: qohnfbyvdvxb # INVALID "c" does not occur in position 3 or 7
5-7 j: abnkhajajjay # VALID "j" occurs in position 5
3-6 p: papvmnnapa   # VALID "p" occurs in position 3
1-5 n: nannnnuuanyn # INVALID "n" occurs in position 1 and position 5
```

In [3]:
valid_pw = 0
for line in lines:
    chmin, chmax, ch, pw = pwre.match(line).groups()
    chmin, chmax = int(chmin), int(chmax)
    if (pw[chmin - 1] == ch) ^ (pw[chmax - 1] == ch):
        valid_pw += 1
        
print(f"{valid_pw} VALID passwords ({len(lines) - valid_pw} INVALID)")

673 VALID passwords (327 INVALID)


**N.B. This is a trick question. The correct answer is that these passwords are all given in plain text which should never ever happen so they are all invalid!**

## Problem 6

In this problem, people are asked to answer yes or no to a list of 26 questions (labeled 'a', 'b', ... 'z'). The responses are recorded by noting down the label of each question for which the response was "yes". For example, responses for three people might look like
```
afuy
rz
aypq
```
The first person has answered yes to questions `a`, `f`, `u` and `y`. The second has answered yes to `r` and `z` etc. We assume that everyone answered yes to at least one question.

The responses of _groups_ of people will be considered with each group being separated by a single blank line. Extending our example from above...
```
afuy
rz
aypq

anqvz

bchklp
bnrt
```
Represents 3 groups. The second group has one member and the third has two members.


### Part 1

In the first part of the problem we are asked to count the number of questions answered "yes" by each group. Duplicate answers are only counted once e.g. For the first group the answer should be 8 (`a` and `y` appear twice but we only count them once.

### Part 1 Answer

Python collections are almost always going to be the foundation of your solution. In this case, sets do what we want to do (throw away duplicate entries).

 * Split entries into groups
 * Find answered questions without duplicates

In [4]:
import re
import requests

try:
    r = requests.get('https://m2pi.syzygy.ca/data/day_6.txt')
    r.raise_for_status()
except Exception as err:
    print(f'Unable to retrieve passwords: {err}')

group_answers = r.text.strip().split('\n\n')
group_answers[:10]

['heqznia\ncipkn\ngvsitwynrxb',
 'auz\nzaeu\nuaz\nzau\nzua',
 'ctqaibd\ntbqlzaywvd\nbqdtcazls\nqtrdvab\nhpbtadq',
 'e\ne\nje\ne\ne',
 'ilzuqnjhrceay\njakzylrnuqcih\nuhyqijldrzwnac',
 'lxtwyiuqerd\nsfdmpjawvolkbzqnih\nldtiewgq',
 'absdepjhctyfzxnivom\nfdvbjnsolpztgywmaihx\ntkashzxmjbydivfnrop\nptnsojahvxmbdzfiy',
 'hd\ndh\nhd\ndh',
 'bup\nktul',
 'rjfzdhowqnystc\nubfkxhagiqmvplw']

In [5]:
total_yes = 0
for group in group_answers:
    total_yes += len(set(group.replace('\n', '')))

print(f"{total_yes} questions answered YES")
    

6249 questions answered YES


### Part 2

In part 2 of this problem, we are told to count the total number of questions for which all participants in a group answered yes. For our example
above...
```
afuy
rz
aypq

anqvz

bchklp
bnrt
```
The total for the first group would be 0 (no letter appears in all three lines). The total for the second group would be 5 because the group size is 1 `a`, `n`, `q`, `v`, `z` all appear once. The total for the third group would be 1 since `b` is the only letter which appears on all lines.

### Part 2 Answer

The collections module in the python standard library has lots of fantastic features. For this problem the `Counter` object behaves like the `set` from above, but keeps track of exactly how many of each object have been seen before.

In [6]:
from collections import Counter

all_yes = 0
for answers in group_answers:
    all_yes += sum([x == len(answers.strip().split('\n')) for x in Counter(answers.replace('\n','')).values()])
    
print(f"{all_yes} questions where all group memebers answered YES")

3103 questions where all group memebers answered YES


## Problem 3: Skyscrapers

See e.g.

 * https://www.chiark.greenend.org.uk/~sgtatham/puzzles/js/towers.html
 * https://www.puzzle-skyscrapers.com/
 
Write a representation of the game in python. You can use any programming paradigm you like but you should offer a method or function to generate puzzles, one to check if they are valid and a way of solving a given puzzle.


In [344]:
import numpy as np

class SkyScraper():
    
    def __init__(self, n=4):
        self.n = n
        self.board = self.generateBoard()
        self.calculateEdges()
    
    def generateBoard(self):
        """
        Generate one valid latin square with arange and roll, e.g. for n=4
        
          1 2 3 4
          4 1 2 3
          3 4 1 2
          2 3 4 1
          
        Now generate permutations of rows and columns.
        """
        board = []
        
        a = np.arange(1, self.n + 1, dtype=np.int)
        for row in range(self.n):
            board.append(np.roll(a, row))

        board = np.array(board)
        
        board = board[np.random.permutation(board.shape[0]), :]
        board = board[:, np.random.permutation(board.shape[1])]
        
        return board

    def validSkyScraper(self):
        return (
            np.all(self.board.sum(axis=0) == (self.n * (self.n + 1) / 2)) and
            np.all(self.board.sum(axis=1) == (self.n * (self.n + 1) / 2)) and
            np.unique(self.board, axis=0).shape == (self.n, self.n) and
            np.unique(self.board, axis=1).shape == (self.n, self.n)
        )
                                
        
    def calculateEdges(self):
        """
        Given a valid board, calculate the number of towers visible from
        each point on the edge of the board
        """
        self.north = np.zeros(self.n, dtype=np.int)
        self.south = np.zeros(self.n, dtype=np.int)
        self.east  = np.zeros(self.n, dtype=np.int)
        self.west  = np.zeros(self.n, dtype=np.int)

        
        for idx in range(self.n):
            self.west[idx] = len(set(np.maximum.accumulate(self.board[idx, :])))
            self.east[idx] = len(set(np.maximum.accumulate(self.board[idx, ::-1])))
            self.north[idx]  = len(set(np.maximum.accumulate(self.board[:, idx])))
            self.south[idx]  = len(set(np.maximum.accumulate(self.board[::-1, idx])))
        
    

    def __repr__(self):
        """
        Display a skyscraper board
        
              |a a a a|
           ---|-------|---
           a  |x x x x| a
           a  |x x x x| a
           a  |x x x x| a
           a  |x x x x| a
           ---|-------|---
              |a a a a|
        
        Spacing is hard coded, will break layout after 9x9
        """
        s  = "   |"
        s += " ".join([f"{buildingsVisible}" for buildingsVisible in self.north])
        s += "|   \n"
        s += "---|" + '-' * (2 * self.n - 1) + "|---\n"
        for row in range(self.n):
            s += f" {self.west[row]} |"
            s += " ".join([f"{x}" for x in self.board[row,:]])
            s += f"| {self.east[row]} \n"
        s += "---|" + '-' * (2 * self.n - 1) + "|---\n"
        s += "   |"
        s += " ".join([f"{buildingsVisible}" for buildingsVisible in self.south])
        s += "|   \n"
        return s

In [350]:
s = SkyScraper(8)

In [351]:
s

   |4 3 3 2 2 2 1 2|   
---|---------------|---
 5 |2 4 3 6 7 5 8 1| 2 
 6 |1 3 2 5 6 4 7 8| 1 
 3 |5 7 6 1 2 8 3 4| 2 
 2 |7 1 8 3 4 2 5 6| 2 
 3 |4 6 5 8 1 7 2 3| 3 
 1 |8 2 1 4 5 3 6 7| 2 
 4 |3 5 4 7 8 6 1 2| 3 
 2 |6 8 7 2 3 1 4 5| 3 
---|---------------|---
   |2 1 2 3 2 4 4 3|   

In [348]:
3+4

7

## Jacobson and Matthews

Turns out the generate method above only explores a tiny portion of the available space of latin squares. Fortunately some kind people at the NSA have described an algorithm for (approximately) uniform random sampling from the full space of Latin Squares via MCMC. Let's do it!

The first insight J&M give is that each $n\times n$ latin square is equivalent to an $n\times n\times n$ incidence matrix. Great, but what the hell is an incidence matrix? In this context it is a 3D marix where each row (in any dimension) contains exactly $n - 1$ zeros and exactly 1 one. To see the correspondence between this and a latin square, think of labeling each entry of the incidence matrix as $M_{i, j, k}$; If $M_{i, j, k} = 1$, then we write $k$ into the $(i, j)$ position of our latin square. This gives us a 1 to 1 correspondance between latin squares and incidence matrices.

More mathematically, we represent a latin square as a function $M$ from the set of ordered triples from $\{1, 2, \ldots, n\}$ to $\{0, 1\}$, such that for any $i, j \in \{ 1, 2, \ldots, n\}$ we have $\sum_{z}M_{i, j, k} = 1$.

Their next big insight is to define a class of "almost" incidence matrices. Given the function definition above, this means that the rows (in all 3 dimensions) which include the $-1$ must include two other $1$ entries so that the sum of all of the values in the row is 1.

These matrices obviously _don't_ correspond to latin squares, but J&M show that together with actual/valid incidence matrices they give us enough freedom tp construct an MCMC which will explore the space of valid incidence matrices (and so latin squares) _with uniform sampling_. To get a random latin square, uniformly distributed in the space of all latin squares, all you need to do is apply these steps and then check that you've landed on a valid incidence matrix. If you haven't you can keep going and check again (there are ~n steps between valid matrices), and these are relatively inexpensive operation.

The key to implementing the MCMC is the transition step. Here is how it works


1. If $M$ is a valid incidence matrix, choose $i, j, k$ with $M_{i,j,k}=0$; if $M$ is
is an almost incidence matrix, start with the unique $i, j, k$ such that $M_{i, j, k} = -1$ 	


2. Let $i', j', k'$ be points such that $M_{i', j, k} = M_{i, j', k} = M_{i, j, k'} = 1$. If $M$ is valid, this choice is unique, if it is almost, there are two pssoble choices for each index.

3. Now increase the value of $M_{i, j, k}$, $M_{i, j', k'}$, $M_{i', j, k'}$ and $M_{i', j', k}$ by 1, and decrease it by 1 $M_{i', j, k}$. $M_{i, j', k}$, $M_{i, j, k'}$ and $M_{i', j', k'}$.

The result will be another actual or almost incidence matrix depending on whether $M_{i', j', k'} = 1$ or $M_{i', j', k'} = 0$.


---

If the incidence matrix is valid then select $i, j, k$ of any $0$ entry in the matrix. In the "almost" incidence case, select the $i, j, k$, of the $-1$ entry. Next choose $i', j', k'$ such that $M_{i', j, k} =  M_{i, j', k} = M_{i, j, k'} = 1$. In the valid case this choice is unique, in the almost case there will be two choices in each dimension, we randomly select between these cases.

Now $i, j, k$ and $i', j', k'$ define a cube within our incidence matrix, the edges of the cube are $\{i, i'\}\times \{j, j'\}\times \{k, k'\}$. $M_{i', j, k} = M_{i, j', k} = M_{i, j, k'} = 1$ by construction and from the incidence condition $M_{i', j', k} = M_{i', j, k'} = M_{i, j', k'} = 0$. If the matrix is valid the $M_{i, j, k} = 0$, if it is almost then $M_{i, j, k} = 0$. Finally, $M_{i', j', k'} = 0$ or $M_{i', j', k'} = 1$.

The transition step is to add 1 to the element $M_{i, j, k}$, subtract one from the element $M_{i′,j′,k′}$, and toggle the remaining entries. If $M_{i′,j′,k′}$ was 1 this will result in a perfect incidence matrix, if not, it will give an almost incidence matrix.

In [564]:
N = 16
r = SkyScraper(N)
t = r.board
t

array([[ 5, 14, 13,  9, 12,  1,  8,  4, 11,  7, 10,  6,  3,  2, 15, 16],
       [10,  3,  2, 14,  1,  6, 13,  9, 16, 12, 15, 11,  8,  7,  4,  5],
       [14,  7,  6,  2,  5, 10,  1, 13,  4, 16,  3, 15, 12, 11,  8,  9],
       [ 9,  2,  1, 13, 16,  5, 12,  8, 15, 11, 14, 10,  7,  6,  3,  4],
       [ 2, 11, 10,  6,  9, 14,  5,  1,  8,  4,  7,  3, 16, 15, 12, 13],
       [ 1, 10,  9,  5,  8, 13,  4, 16,  7,  3,  6,  2, 15, 14, 11, 12],
       [15,  8,  7,  3,  6, 11,  2, 14,  5,  1,  4, 16, 13, 12,  9, 10],
       [ 4, 13, 12,  8, 11, 16,  7,  3, 10,  6,  9,  5,  2,  1, 14, 15],
       [11,  4,  3, 15,  2,  7, 14, 10,  1, 13, 16, 12,  9,  8,  5,  6],
       [ 8,  1, 16, 12, 15,  4, 11,  7, 14, 10, 13,  9,  6,  5,  2,  3],
       [16,  9,  8,  4,  7, 12,  3, 15,  6,  2,  5,  1, 14, 13, 10, 11],
       [ 7, 16, 15, 11, 14,  3, 10,  6, 13,  9, 12,  8,  5,  4,  1,  2],
       [ 3, 12, 11,  7, 10, 15,  6,  2,  9,  5,  8,  4,  1, 16, 13, 14],
       [12,  5,  4, 16,  3,  8, 15, 11,  2, 14,  1,

With a bit of indexing magic we can turn an existing latin square into a valid incidence matrix

In [565]:
xy = np.mgrid[0:N, 0:N]
idx = np.array([xy[0], xy[1], r.board - 1])

In [566]:
v = np.zeros((N, N, N), dtype=np.int)

v[idx[2], idx[0], idx[1]] = 1
v

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 1, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [567]:
i, j, k = np.unravel_index(np.argmin(v, axis=None), v.shape)

In [568]:
def validIncidenceMatrix(v):
    return (
        (len(v.shape) == 3) and 
        (v.shape[0] == v.shape[1] == v.shape[2]) and
        np.all((v.sum(axis=0) == np.ones_like(v))) and
        np.all((v.sum(axis=1) == np.ones_like(v))) and
        np.all((v.sum(axis=2) == np.ones_like(v)))
    )

def properIncidenceMatrix(v):
    u = np.unique(v)
    return (
        validIncidenceMatrix(v) and
        (len(u) == 2) and
        (min(u) == 0)
    )


In [569]:
def step(v):
    if properIncidenceMatrix(v):
        # Find indices of a zero
        while True:
            # Pick a random coordinate until we get a zero
            i, j, k = np.random.randint(N, size=3)
            if v[i, j, k] == 0: break
    
        i1 = np.argmax(v[:, j, k])
        j1 = np.argmax(v[i, :, k])
        k1 = np.argmax(v[i, j, :])
    
    else:
        # find indices of the -1 entry
        i, j, k = np.unravel_index(np.argmin(v, axis=None), v.shape)
        
        # Each line passing through (i, j, k) has two 1 values
        # find them both and randomly select one.
        i1 = np.random.choice(np.where(v[:, j, k] == 1)[0])
        j1 = np.random.choice(np.where(v[i, :, k] == 1)[0])
        k1 = np.random.choice(np.where(v[i, j, :] == 1)[0])
        
    v[i,  j,  k ] += 1
    v[i1, j1, k ] += 1
    v[i1, j,  k1] += 1
    v[i,  j1, k1] += 1
    
    v[i1, j1, k1] -= 1
    v[i1, j,  k ] -= 1
    v[i,  j1, k ] -= 1
    v[i,  j,  k1] -= 1



In [570]:
def next(v):
    iteration = 0
    while iteration < N * N * N or not properIncidenceMatrix(v):
        iteration += 1
        step(v)


In [571]:
next(v)
v

array([[[0, 0, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 1],
        ...,
        [0, 0, 0, ..., 